# Projeto 1 - Ciência dos Dados

Nome: Raphael Lahiry de Barros

Nome: Florencia Averame Kramer

# O que temos que fazer?
precisamos criar um dataframe para cada relevância   

dentro desses dataframes separar as frases em palavras    

analisar a freq relativa de cada palavra --> probabilidade daquela palavra pertencer àquela relevância    

quando quisermos avaliar o teste--> analisamos a probabilidade de cada palavra de uma frase pertencer a uma relevância (usar o smoothing -- aula 8 -- para as palavras que talvez não existam em certas categorias)  

com isso, para cada palavra teremos a probabilidade de que esta pertença a cada relevância  
a relevância com maior probabilidade será a daquela palavra

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [62]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [63]:
import os

filename = 'ps4.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo ps4.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [64]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,@odonodocabare_ comprei um tablet dia desses e...,0.0
1,@encanadorchapad o negócio é o ps4 slim. peque...,4.0
2,rt @ps3brasil: tohu é anunciado para ps4; deta...,2.0
3,dividindo essa semana entre dois jogos no modo...,0.0
4,@leobruto @fridaygus @douglasxavier8 @ocerveje...,0.0


In [102]:
train['Treinamento'] = train['Treinamento'].astype('str')

In [67]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@chaehobbit simmm sou apaixonada em jogos assi...,1.0
1,"@pimps_junin @joao_almirante atleta nato , pod...",1.0
2,@isa_boechatt kkkk vc joga onde no xbox ps4 ou...,1.0
3,@vezguu eu sou ps4,1.0
4,path of exile (ps4) - farmando e fazendo o atl...,0.0


In [68]:
test['Teste'] = test['Teste'].astype('str')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é o playstation 4. Consideramos relevantes as mensagens que falavam sobre questões técnicas do produto e sobre satisfação, sejam positivas ou negativas. Consideramos irrelevantes as mensagens que só mencionavam o nosso produtos nas '#' ou que eram opiniões pessoais que não agregavam na nossa análise.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [127]:
#função que apaga os caracteres especiais e as menções de usuários
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    #filtro que remove as menções à usuários com @
    filtro = filter(lambda x:x[0]!='@', text_subbed.split())
    juncao = " ".join(filter(lambda x:x[0]!='@', text_subbed.split()))
    
    return juncao
    

In [128]:
#aplicando a função cleanup na base treinamento e deixando tudo em letra minúscula
train['Treinamento'] = train['Treinamento'].apply(cleanup)
train['Treinamento'] = train['Treinamento'].str.lower()

In [129]:
#ligando a relevância à um número
muito_relevante = train['Relevância'] == 4
relevante = train['Relevância'] == 3
neutro = train['Relevância'] == 2
irrelevante = train['Relevância'] == 1
muito_irrelevante = train['Relevância'] == 0

In [130]:
train.head()

,Treinamento,Relevância
0,comprei um tablet dia desses e to bem feliz pq...,0.0
1,o negócio é o ps4 slim pequeno e silencioso ec...,4.0
2,rt tohu é anunciado para ps4 detalhes e gamepl...,2.0
3,dividindo essa semana entre dois jogos no modo...,0.0
4,sentiu kkkkk só fiz uma pergunta bem simples s...,0.0


In [131]:
df_muito_relevante = train.loc[muito_relevante, :]
df_relevante = train.loc[relevante, :]
df_neutro = train.loc[neutro, :]
df_irrelevante = train.loc[irrelevante, :]
df_muito_irrelevante = train.loc[muito_irrelevante, :]

In [132]:
df_muito_relevante

,Treinamento,Relevância
1,o negócio é o ps4 slim pequeno e silencioso ec...,4.0
6,e como eu sempre digo veremos no lançamento se...,4.0
16,a diferença do ps4 base184tf e one x não é de ...,4.0
22,se tu joga num ps4 tu n sabe oqq é ser mal oti...,4.0
35,sim bro mas aconteceu o mesmo na ps3 nos últim...,4.0
44,fiquei numa agonia de pegar um ps4 de volta qn...,4.0
56,a minha é net de 120mb tambem no computador eu...,4.0
106,no pc é diferente amigão o hardware do meu pc ...,4.0
152,ps4 tem vantatem em ssd e xbox tem vantagem em...,4.0
174,pior é gente querendo comparar com o lançament...,4.0


In [133]:
df_relevante

,Treinamento,Relevância
7,tô quase terminando radiant historia de novo e...,3.0
8,eles publicaram outra matéria falando que o pr...,3.0
9,queria pra ps4,3.0
14,de uma coisa eu tenho certeza alexandre se arr...,3.0
24,rt 13 segundos de atualização no ps4 para depo...,3.0
...,...,...
712,vdd fortnite n é mal otimizado é pessimamente ...,3.0
724,a sony acabou de falar que vai sair do br o es...,3.0
728,se minha experiência e observações pessoais va...,3.0
731,o problema é que a nintendo nunca muda o preço...,3.0


In [134]:
df_neutro

,Treinamento,Relevância
2,rt tohu é anunciado para ps4 detalhes e gamepl...,2.0
12,ps4 foi fabricado no brasil até 2017 e teve su...,2.0
17,sim o time do ps4 pode ir para o ps5 sem probl...,2.0
19,ps4 25 mil reais 😡😡😡😡 notebook 5 mil reais 😍😍😍🥰,2.0
28,tava na fila do continente e reparei que agora...,2.0
...,...,...
734,rt uma vergonha o jogo ser lançado a 0000 do d...,2.0
736,amiga o playstation não é fabricado no brasil ...,2.0
741,eu sinceramente gosto mais do controle de ps4 ...,2.0
745,eu nunca acho alguém do ps4 😕,2.0


In [135]:
df_irrelevante

,Treinamento,Relevância
13,o menino q chora pelo ps4 q o pai não pode com...,1.0
15,comprei um kit da ice suave o preço eu pegava ...,1.0
20,casar com o ps3 pq o ps4 e ps5 eu n tenho,1.0
26,playstation lança novo anúncio para celebrar n...,1.0
32,rt ghostrunner será lançado em 27 de outubro n...,1.0
...,...,...
743,enganado com o ps4 pro kkkk,1.0
744,jogo no ps4 mas seria uma honra jogar contigo ...,1.0
746,as publishers pixelheart e just for games junt...,1.0
747,eu so n posso mandar print mas pelo oq eu sei ...,1.0


In [136]:
df_muito_irrelevante

,Treinamento,Relevância
0,comprei um tablet dia desses e to bem feliz pq...,0.0
3,dividindo essa semana entre dois jogos no modo...,0.0
4,sentiu kkkkk só fiz uma pergunta bem simples s...,0.0
5,kkkkk tá explicado pq o abreu entrou me pergun...,0.0
10,cria uma nova conta no ps4 kkkk so os pro player,0.0
...,...,...
727,rt as paredes estão mudando de lugar de novo 🐍...,0.0
729,kurtzflorio e ask só os cria do ps4,0.0
738,jogar ps4,0.0
740,bateu de cabeça fallguys facebookgaming stream...,0.0


## Separando cada coluna de relevância em listas de palavras

In [152]:
palavras_muito_relevantes = []
lista_vrw = []
for vrw in df_muito_relevante['Treinamento']:
    palavras_muito_relevantes.append(vrw.split(' '))
for vrl in palavras_muito_relevantes:
    lista_vrw += vrl

In [166]:
palavras_relevantes = []
lista_rw = []
for rw in df_relevante['Treinamento']:
    palavras_relevantes.append(rw.split(' '))
for rl in palavras_relevantes:
    lista_rw += rl

In [154]:
palavras_neutras = []
lista_nw = []
for nw in df_neutro['Treinamento']:
    palavras_neutras.append(nw.split(' '))
for nl in palavras_neutras:
    lista_nw += nl

In [155]:
palavras_irrelevantes = []
lista_iw = []
for iw in df_irrelevante['Treinamento']:
    palavras_irrelevantes.append(iw.split(' '))
for il in palavras_irrelevantes:
    lista_iw += il

In [156]:
palavras_muito_irrelevantes = []
lista_viw = []
for viw in df_muito_irrelevante['Treinamento']:
    palavras_muito_irrelevantes.append(viw.split(' '))
for vil in palavras_muito_irrelevantes:
    lista_viw += vil

## Transformando as listas de palavras de cada relevância em pd.Series

In [176]:
serie_muito_relevante = pd.Series(lista_vrw)
serie_muito_relevante.value_counts()

o            46
ps4          41
e            33
que          30
do           27
             ..
para          1
tão           1
rolar         1
aleatório     1
morreu        1
Length: 518, dtype: int64

In [170]:
serie_relevante = pd.Series(lista_rw)
serie_relevante.value_counts()

ps4        129
o          114
que         75
de          72
e           67
          ... 
normal       1
boas         1
gosto        1
loucura      1
faixa        1
Length: 1090, dtype: int64

In [171]:
serie_neutro = pd.Series(lista_nw)
serie_neutro.value_counts()

ps4          159
o            118
de            84
e             83
no            75
            ... 
usado          1
ia             1
epicgames      1
features       1
sincera        1
Length: 1232, dtype: int64

In [172]:
serie_irrelevante = pd.Series(lista_iw)
serie_irrelevante.value_counts()

ps4                      221
o                        117
de                       109
e                        103
a                         77
                        ... 
sim                        1
photo                      1
produto                    1
https//tco/bhvr8xxr69      1
just                       1
Length: 1415, dtype: int64

In [173]:
serie_muito_irrelevante = pd.Series(lista_viw)
serie_muito_irrelevante.value_counts()

ps4                      186
de                        79
e                         73
o                         69
no                        53
                        ... 
https//tco/tsrekb291y      1
astro                      1
https//tco/p89psumgmd      1
necessita                  1
hahah                      1
Length: 1392, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [31]:
test['Teste'] = test['Teste'].apply(cleanup)
test['Teste'] = test['Teste'].str.lower()

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**